In [1]:
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from core.elements.dormitory import Dormitory
from core.elements.student import Student
from core.elements.floor import Floor
from core.elements.room import Room

from core.engine.state_processor import StateProcessor

from frontend.pygame_manager import PygameManager

In [ ]:
dormitory = Dormitory()
state_processor = StateProcessor(dormitory)

pygame_manager = PygameManager()

# Main game loop
clock = pygame.time.Clock()
tick = 2
running = True
paused = True
manual = False
time = 24

is_exam_season = False

pygame_manager.draw(dormitory, time, state_processor.exam_difficulty)
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                if manual:
                    state_processor.iterate(is_exam_season)
                    time += 1
                    break
                paused = not paused
            else:
                running = pygame_manager.handle_keydown(event, dormitory)
        elif not paused and not manual:
            state_processor.iterate(is_exam_season)
            time += 1
            break

    pygame_manager.draw(dormitory, time, state_processor.exam_difficulty)
    is_exam_season = (time // 24) % 7 == 0
    print("time: ", time )
    clock.tick(tick)
    #pygame.time.delay(2500)
    

pygame.quit()

Drawing took 0.08156299591064453 seconds
time:  24
time:  24
Drawing took 0.06862807273864746 seconds
time:  24
Drawing took 0.06429481506347656 seconds
time:  25
Drawing took 0.06660676002502441 seconds
time:  26
Drawing took 0.0662393569946289 seconds
time:  27
Drawing took 0.0662832260131836 seconds
time:  28
time:  28
Drawing took 0.06637334823608398 seconds
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
time:  29
